In [1]:
"""xarray/dask-compatible finite differencing functions"""
import dask.array as darray
import numpy as np
import xarray as xr

from scipy.ndimage import correlate1d
from sympy.calculus import finite_diff_weights
#from xarray.core import computation

In [2]:
def centered_diff_weights(accuracy, spacing=1.):
    """Compute the weights of a central difference approx. to derivative

    Does so for an arbitrary even-valued order of accuracy.

    Parameters
    ----------
    accuracy : int
        Order of accuracy of approximation (must be even)
    spacing : float
        Uniform spacing between the points (default = 1.)

    Returns
    -------
    list of finite difference weights
    """
    if accuracy % 2:
        raise ValueError('Can only generate centered difference stencil '
                         'for an even-valued order of accuracy.  Got '
                         '{}'.format(accuracy))
    domain = np.arange(0., accuracy + 1.) * spacing
    center = (accuracy / 2.) * spacing
    return finite_diff_weights(1, domain, center)[1][-1]

In [3]:
def forward_diff_weights(accuracy, spacing=1.):
    """Compute the weights of a forward difference approx. to derivative

    Does so for an arbitrary order of accuracy.

    Parameters
    ----------
    accuracy : int
        Integer-valued order of accuracy of approximation
    spacing : float
        Uniform spacing between points (default = 1.)

    Returns
    -------
    list of finite difference weights
    """
    domain = np.arange(0., accuracy + 1.) * spacing
    return finite_diff_weights(1, domain, domain[0])[1][-1]

In [4]:
def backward_diff_weights(accuracy, spacing=1.):
    """Compute the weights of a backward difference approx. to derivative

    Does so for an arbitrary order of accuracy.

    Parameters
    ----------
    accuracy : int
        Integer-valued order of accuracy of approximation
    spacing : float
        Uniform spacing between points (default = 1.)

    Returns
    -------
    list of finite difference weights
    """
    domain = np.arange(0., accuracy + 1.) * spacing
    return finite_diff_weights(1, domain, domain[-1])[1][-1]

In [5]:
def xcorrelate1d(da, dim, kernel, **kwargs):
    """Apply ``correlate1d`` along a given dimension

    Parameters
    ----------
    da : xr.DataArray
        DataArray
    dim : str
        Dimension name
    kernel : array-like
        1D sequence of numbers (i.e. correlation weights)
    **kwargs
        Keyword arguments to supply to ``scipy.ndimage.filters.correlate1d``

    Returns
    -------
    xr.DataArray
    """
    def apply_corr(arr, **kwargs):
        #
        # If not a Dask array
        #
        if not isinstance(arr, darray.core.Array):
            return correlate1d(arr, **kwargs)
        else:
            origin = kwargs.get('origin', 0)
            depth = int(len(kwargs['weights']) / 2 + np.abs(origin))
            axis = len(arr.shape) - 1

            if kwargs['mode'] != 'wrap':
                # TODO: Don't hard-code periodic boundary conditions
                raise NotImplementedError(
                    'xdiff currently only supports periodic boundary '
                    'conditions on dask arrays')
            
            return darray.map_overlap(correlate1d,arr,
                                            depth={axis: depth},
                                            boundary={axis: 'periodic'},
                                            **kwargs)

    return xr.apply_ufunc(apply_corr, da, input_core_dims=[[dim]],
                                   output_core_dims=[[dim]],
                                   kwargs=dict(weights=kernel, **kwargs),
                                   dask='allowed')


In [6]:
def xdiff(da, dim, method='centered', accuracy=2, spacing=1, mode='wrap'):
    """Compute the derivative to an arbitrary order of accuracy

    Assumes uniform grid spacing.

    Parameters
    ----------
    da : xr.DataArray
        DataArray
    dim : str
        Dimension name to perform derivative along
    method : str
        Options are 'centered', 'backward', and 'forward'
    accuracy : int
        Order of accuracy of approximation
    spacing : float
        Grid spacing
    mode : str
        How to handle boundary; options are same as those that can be passed to
        correlate1d

    Returns
    -------
    xr.DataArray
    """
    if method == 'centered':
        weights = centered_diff_weights(accuracy, spacing)
        origin = 0
    elif method == 'forward':
        weights = forward_diff_weights(accuracy, spacing)
        origin = -int(np.ceil(accuracy / 2.))
    elif method == 'backward':
        weights = backward_diff_weights(accuracy, spacing)
        origin = int(np.floor(accuracy / 2.))
    else:
        raise ValueError('Invalid differencing method '
                         'specified: {}'.format(method))
    return xcorrelate1d(da, dim, weights, origin=origin, mode=mode)

In [7]:
def xgradient(da, dim, accuracy=2, spacing=1.):
    """Arbitrary even-order extension of np.gradient for DataArrays.

    Meant for computing approximations for derivatives in a non-peridiodic
    setting.  Uses centered differencing in the interior, and forward and
    backward differencing on the left and right edges respectively.  Currently
    only supports operations along a single axis (though operations along
    multiple axes can be done with repeated calls to ``xgradient``).

    Parameters
    ----------
    da : xr.DataArray
        DataArray
    dim : str
        Name of dimension
    accuracy : int
        Even order of accuracy of differencing approximation
    spacing : float
        Distance between points (must be uniform)

    Returns
    -------
    xr.DataArray
    """
    interior = xdiff(
        da, dim, method='centered',
        accuracy=accuracy, spacing=spacing).isel(
            **{dim: slice(accuracy, -accuracy)})

    left = da.isel(**{dim: slice(None, 2 * accuracy)})
    left = left.chunk({dim: left.sizes[dim]})
    left = xdiff(
        left, dim, method='forward',
        accuracy=accuracy, spacing=spacing).isel(
            **{dim: slice(None, accuracy)})

    right = da.isel(**{dim: slice(-2 * accuracy, None)})
    right = right.chunk({dim: right.sizes[dim]})
    right = xdiff(
        right, dim, method='backward',
        accuracy=accuracy, spacing=spacing).isel(
            **{dim: slice(-accuracy, None)})

    return xr.concat([left, interior, right], dim=dim).chunk(chunks=da.chunksizes)

In [8]:
def curl(m,n,p,dx,dy,dz,accuracy):
    """ Compute curl of a 3D vector field.
    
    Parameters
    ----------
    m : xr.DataArray
        X component of velocity
    n : xr.DataArray
        X component of velocity
    p : xr.DataArray
        X component of velocity
    dx : Scalar
        Sample spacing along X direction
    dy : Scalar
        Sample spacing along Y direction
    dz : Scalar
        Sample spacing along Z direction
    accuracy : int
        Even order of accuracy of differencing approximation
        
    Returns wx,wy,wz
        X,Y,Z components of curl
    """
    
    aux1 = xgradient(p, p.dims[1], accuracy=accuracy, spacing=dy)
    aux2 = xgradient(n, n.dims[0], accuracy=accuracy, spacing=dz)
    outx = aux1-aux2
    
    aux1 = xgradient(m, m.dims[0], accuracy=accuracy, spacing=dz)
    aux2 = xgradient(p, p.dims[2], accuracy=accuracy, spacing=dx)
    outy = aux1-aux2
    
    aux1 = xgradient(n, n.dims[2], accuracy=accuracy, spacing=dx)
    aux2 = xgradient(m, m.dims[1], accuracy=accuracy, spacing=dy)
    outz = aux1-aux2
    
    return outx, outy, outz
        

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
x = np.linspace(0., 2. * np.pi, 100, endpoint=False)
dx = x[1] - x[0]
test = xr.DataArray(np.sin(x), coords=[x], dims=['x'])

In [ ]:
f = xdiff(test, 'x', accuracy=2, method='centered', spacing=dx)

In [ ]:
test.plot(label='input')
xdiff(test, 'x', accuracy=2, method='centered', spacing=dx).plot(label='result (centered)')
xdiff(test, 'x', accuracy=2, method='forward', spacing=dx).plot(label='result (forward)')
#xdiff(test, 'x', accuracy=2, method='backward', spacing=dx).plot(label='result (backward)')
plt.gca().legend(loc='lower left')

In [ ]:
x = np.linspace(0., 2. * np.pi, 100, endpoint=False)
dx = x[1] - x[0]
test = xr.DataArray(np.sin(x), coords=[x], dims=['x']).chunk({'x': 10})
test.data

In [ ]:
test.plot(label='input')
xdiff(test, 'x', accuracy=2, method='centered', spacing=dx).plot(label='result (centered)')
xdiff(test, 'x', accuracy=2, method='forward', spacing=dx).plot(label='result (forward)')
xdiff(test, 'x', accuracy=2, method='backward', spacing=dx).plot(label='result (backward)')
plt.gca().legend(loc='lower left')

In [ ]:
x = np.linspace(0., np.pi, 100, endpoint=False)  # Halt at np.pi (rather than 2 * np.pi)
dx = x[1] - x[0]
test = xr.DataArray(np.sin(x), coords=[x], dims=['x']).chunk({'x': 10})


In [ ]:
test.plot(label='input')
xgradient(test, 'x', accuracy=8, spacing=dx).plot(label='result')
plt.gca().legend(loc='lower left')

In [ ]:
x = np.linspace(0., 2. * np.pi, 100, endpoint=False)
dx = x[1] - x[0]
y = xr.DataArray(np.arange(10), coords=[np.arange(10)], dims=['y'])
z = xr.DataArray(np.arange(12, 20), coords=[np.arange(12, 20)], dims=['z'])
test = xr.DataArray(np.sin(x), coords=[x], dims=['x'])

In [ ]:
test3d, _ = xr.broadcast(test, y * z)
test3d = test3d.transpose('y', 'x', 'z').chunk({'x': 10, 'y': 1})
test3d

In [36]:
n=512
c=64
x1d = np.linspace(0., 1., n)
x = np.broadcast_to(x1d, (n,n,n))

y1d = np.linspace(0., 1., n)
y = np.broadcast_to(y1d[:,None], (n,n,n))
        
z1d = np.linspace(0., 1., n)

vx = y*y*y - 9*y
vy = x*x*x - 9*x
vz = np.zeros((n,n,n))

vx = xr.DataArray(vx, coords=[z1d, y1d, x1d], dims=['z', 'y', 'x']).chunk((c,c,c))
vy = xr.DataArray(vy, coords=[z1d, y1d, x1d], dims=['z', 'y', 'x']).chunk((c,c,c))
vz = xr.DataArray(vz, coords=[z1d, y1d, x1d], dims=['z', 'y', 'x']).chunk((c,c,c))


In [37]:
ds = vx.to_dataset(name = 'vx')
ds['vy'] = vy
ds['vz'] = vz






In [38]:
%%time
wx,wy,wz =  curl(vx,vy,vz,x1d[1]-x1d[0],y1d[1]-y1d[0],z1d[1]-z1d[0],accuracy=6)
wx.compute();
wy.compute();
wz.compute();
f=1

CPU times: user 21.6 s, sys: 5.04 s, total: 26.6 s
Wall time: 14 s


In [20]:
wxtrue = np.zeros((n,n,n))
wytrue = np.zeros((n,n,n))
wztrue = (3*x*x) - (3*y*y)

In [21]:

#wxerrmax = darray.max(darray.absolute(wx-wxtrue))
wxerrmax = darray.absolute(wx-wxtrue)
f = wxerrmax.max()
print(float(f))
wyerrmax = darray.absolute(wy-wytrue)
f = wyerrmax.max()
print(float(f))
wzerrmax = darray.absolute(wz-wztrue)
f = wzerrmax.max()
print(float(f))
#print(int(wxerrmax.argmax(dim=["x", "y", "z"])['x']))
#print(int(wxerrmax.argmax(dim=["x", "y", "z"])['y']))
#print(int(wxerrmax.argmax(dim=["x", "y", "z"])['z']))

1.4551915228366852e-11
1.4551915228366852e-11
1.3320899938662478e-11


In [28]:
ds['wx'] = wx
ds['wy'] = wy
ds['wz'] = wz

ds['x'].attrs['axis'] = 'X'
ds['y'].attrs['axis'] = 'Y'
ds['z'].attrs['axis'] = 'Z'

In [27]:
ds.to_netcdf('/tmp/foo.nc', mode='w')

In [16]:
np.amax(vz.values)

0.0

In [ ]:
a = np.arange(6).reshape(2,3)
a

In [ ]:
a[1,:]